# YData Quality - Bias & Fairness Tutorial
Time-to-Value: 3 minutes

This notebook provides a tutorial for the ydata_quality package module on Bias & Fairness .

**Structure:**

1. A bias and fairness introduction
2. Load example dataset
3. Instantiate the Data Quality engine
4. Run the quality checks
5. Assess the warnings
6. (Extra) Detailed overview

## Bias and Fairness
As data is increasingly used for automated decision making with heavy impact on individual lifes, a thorough analysis of data quality encompasses an understanding of the inherent biases embedded in the datasets that can cause different treatments based on sensitive attributes. There are ethical and legal obligations for Data Scientists to develop applications which are unbiased and fair.

### Definitions
We consider _fairness_ to be the absence of differentiated treatment (assistive or punitive) based on sensitive attributes. _Fairness_ can also be thought of as the absence of unjustified basis for differentiated treatment.

We consider _sensitive attributes (a.k.a. sensitive features)_ as personal details for which there are legal and ethical obligations for which not to differentiate the treatment based on.

We consider _bias_ as a systematic, non-neglectable treatment which is differentiated towards a specific sub-group of individuals.

### Remarks
- The absence of sensitive attributes in a data application does not guarantee fairness by default. 
- Biases in data can be originated from multiple sources: sample (e.g. fair representation of multiple groups), label (e.g. how the outcome was defined), machine learning pipeline (e.g. how data is diggested), application (e.g. how an application is deployed). Not all biases are available during data quality analysis but it is important to keep them in mind from the beginning.



**References**
- Chapter 11 Bias and Fairness | Big Data and Social Science [(link)](https://textbook.coleridgeinitiative.org/chap-bias.html)
- Fairness and Machine Learning - Limitations and Opportunities [(link)](https://fairmlbook.org/)
- Fairness Tutorial - Moritz Hardt - MLSS 2020, Tübingen [(link)](https://www.youtube.com/watch?v=Igq_S_7IfOU)

In [1]:
import pandas as pd
from ydata_quality.bias_fairness import BiasFairness

## Load the example dataset
The "Adult Data Set" (a.k.a. "Census Income") contains a set of records to predict whether an individual's income exceeds $50K/yr, based on census data. 

In [2]:
# This is the DataFrame used in the demo from GE tutorials
df = pd.read_csv('../datasets/transformed/census_10k.csv')
df.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income,workclass2
0,57,Private,132704,Masters,14,Separated,Prof-specialty,Not-in-family,White,Male,10520,0,32,United-States,>50K,Private
1,25,Private,321205,Bachelors,13,Never-married,Exec-managerial,Not-in-family,White,Male,4101,0,35,United-States,<=50K,Private
2,25,Private,38090,Bachelors,13,Never-married,Sales,Not-in-family,White,Male,0,0,44,United-States,<=50K,Private
3,54,Private,29909,11th,7,Married-civ-spouse,Other-service,Wife,White,Female,0,0,43,United-States,<=50K,Private
4,43,Private,347934,HS-grad,9,Separated,Sales,Not-in-family,White,Female,0,0,30,United-States,<=50K,Private


## Create the engine
Each engine contains the checks and tests for each suite.

In [3]:
bf = BiasFairness(df=df, sensitive_features=['race', 'sex'], label='income', random_state=42)

### Full Evaluation
The easiest way to assess the data quality analysis is to run `.evaluate()` which returns a dictionary with outputs of operation performed.

In [4]:
results = bf.evaluate()

Warnings:
	TOTAL: 2 warning(s)
	Priority 2: 2 warning(s)


Priority 2 - usage allowed, limited human intelligibility:
	* [BIAS&FAIRNESS - PROXY IDENTIFICATION] Found 1 feature pairs of correlation to sensitive attributes with values higher than defined threshold (0.5).
	* [BIAS&FAIRNESS - SENSITIVE ATTRIBUTE REPRESENTATIVITY] Found 2 values of 'race' sensitive attribute with low representativity in the dataset (below 1.00%).



After running the data quality checks, you can check the warnings for each individual test of the Bias & Fairness module. The warnings are sorted by priority and have additional details that can provide better insights for Data Scientists.

In [5]:
bias_fairness_warnings = bf.get_warnings()
bias_fairness_warnings

[QualityWarning(category='Bias&Fairness', test='Proxy Identification', description='Found 1 feature pairs of correlation to sensitive attributes with values higher than defined threshold (0.5).', priority=<Priority.P2: 2>, data=features
 relationship_sex    0.650656
 Name: association, dtype: float64),
 QualityWarning(category='Bias&Fairness', test='Sensitive Attribute Representativity', description="Found 2 values of 'race' sensitive attribute with low representativity in the dataset (below 1.00%).", priority=<Priority.P2: 2>, data=Other                 0.0092
 Amer-Indian-Eskimo    0.0090
 Name: race, dtype: float64)]

## Full Test Suite
In this section, you will find a detailed overview of the available tests in the data expectations module of ydata_quality. These are all run with the `evaluate` method, which centralizes input arguments and produces specific outputs in the returned results dictionary, structured by test.

In [6]:
# Results object structure
list(results.keys())

['performance_discrimination',
 'proxy_identification',
 'sensitive_predictability',
 'sensitive_representativity']

### Performance Discrimination
The "Performance Discrimination" inspects the disparities in the classifying performance across values of sensitive attributes. A model is trained on the full data but the performance metrics are broken down per each sub-group, enabling the Data Scientist to understand the performance of a baseline classifier throughout each partition of sub-group data.

In [7]:
performances = bf.performance_discrimination()
performances

{'race': Amer-Indian-Eskimo    1.000000
 Asian-Pac-Islander    0.510216
 Other                 1.000000
 Black                 0.639949
 White                 0.562961
 dtype: float64,
 'sex': Female    0.526722
 Male      0.591663
 dtype: float64}

### Proxy Identification
The "Proxy Identification" test aims to detect associations between sensitive and non-sensitive attributes in the data, to signal Data Scientists for unwanted proxies. The leakage of protected features by non-protected ones may give Data Scientists a false confidence that by disregarding sensitive attributes that they will not be available in the data (directly or indirectly).

In [8]:
bf.proxy_identification(th=0.2)

features
relationship_sex       0.650656
marital-status_sex     0.459833
occupation_sex         0.426367
native-country_race    0.402390
hours-per-week_sex     0.235445
income_sex             0.208361
Name: association, dtype: float64

## Sensitive Attributes Predictability
The "Sensitive Attributess Predictability"  test builds a predictive model with sensitive attributes as targets to grasp how easy it is to build a baseline that can predict protected features based on non-protected ones. Similar to the "Proxy Identification" mechanism with more complex relationships (based on a model, not on association measures). The leakage of protected features by non-protected ones may give Data Scientists a false confidence that by disregarding sensitive attributes that they will not be available in the data (directly or indirectly).

The performance metric values are calculated as a ratio of _(real - min)/(max - min)_. The best performance (max) is defined as a perfect prediction, where the predictions equal the true outcomes. The baseline performance (min) is defined as the performance achieved by a naive model, where the predictions are the mode for classification or mean for regression.

The values returned by the test indicate the percentage of achievable performance achieved by a baseline model trained to predict a sensitive attribute.

In [9]:
sens_pred = bf.sensitive_predictability()
sens_pred

race     0.12168
sex     0.249346
dtype: object

In [10]:

bf.sensitive_predictability(adjusted_metric=False)

race    0.049889
sex     0.624673
dtype: object

## Sensitive Attributes Representativity
The "Sensitive Attributes Representativity" calculates the distribution of categorical features to assess if any sub-group of a sensitive attribute is underrepresented. It is meant for Data Scientists to validate the data for sampling bias, i.e. the systematic over/under representation of some members of a population in relation to the others. 

In [11]:
bf.sensitive_representativity()

{'race': White                 0.8537
 Black                 0.0978
 Asian-Pac-Islander    0.0303
 Other                 0.0092
 Amer-Indian-Eskimo    0.0090
 Name: race, dtype: float64,
 'sex': Male      0.6657
 Female    0.3343
 Name: sex, dtype: float64}